# Feature Selection

In [20]:
import numpy as np # Linear algebra
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt  # Matlab-style plotting
# Make sure plot shows immediately
%matplotlib inline 
import seaborn as sns # Library for plotting
color = sns.color_palette()
sns.set_style('darkgrid')

In [21]:
df_member = pd.read_csv("./processed/Members.csv", index_col=["MemberID"])

In [22]:
df_member

,AgeAtFirstClaim,Sex_M,Sex_NaN
MemberID,,,
14723353,75.0,1,0
75706636,75.0,1,0
17320609,75.0,1,0
69690888,45.0,1,0
33004608,5.0,1,0
...,...,...,...
99711514,45.0,0,0
31690877,55.0,0,0
9519985,35.0,0,0


In [23]:
df_claim_count_y1 = pd.read_csv("./processed/ClaimCount_Y1.csv", index_col=["MemberID"])
df_claim_count_y2 = pd.read_csv("./processed/ClaimCount_Y2.csv", index_col=["MemberID"])

In [24]:
df_char_y1 = pd.read_csv("./processed/CharlsonIndex_Count_Y1.csv", index_col=["MemberID"])
df_char_y2 = pd.read_csv("./processed/CharlsonIndex_Count_Y2.csv", index_col=["MemberID"])

In [25]:
df_drug_y1 = pd.read_csv("./processed/DrugCount_Y1.csv", index_col=["MemberID"])
df_drug_y2 = pd.read_csv("./processed/DrugCount_Y2.csv", index_col=["MemberID"])

In [26]:
df_dsfs_y1 = pd.read_csv("./processed/DSFS_Count_Y1.csv", index_col=["MemberID"])
df_dsfs_y2 = pd.read_csv("./processed/DSFS_Count_Y2.csv", index_col=["MemberID"])

In [27]:
df_lab_y1 = pd.read_csv("./processed/LabCount_Y1.csv", index_col=["MemberID"])
df_lab_y2 = pd.read_csv("./processed/LabCount_Y2.csv", index_col=["MemberID"])

In [28]:
df_los_y1 = pd.read_csv("./processed/LengthOfStay_Count_Y1.csv", index_col=["MemberID"])
df_los_y2 = pd.read_csv("./processed/LengthOfStay_Count_Y2.csv", index_col=["MemberID"])

In [29]:
df_pay_max_y1 = pd.read_csv("./processed/PayDelayMax_Y1.csv", index_col=["MemberID"])
df_pay_max_y2 = pd.read_csv("./processed/PayDelayMax_Y2.csv", index_col=["MemberID"])
df_pay_mean_y1 = pd.read_csv("./processed/PayDelayMean_Y1.csv", index_col=["MemberID"])
df_pay_mean_y2 = pd.read_csv("./processed/PayDelayMean_Y2.csv", index_col=["MemberID"])

In [30]:
df_pcp_y1 = pd.read_csv("./processed/PCP_Y1.csv", index_col=["MemberID"])
df_pcp_y2 = pd.read_csv("./processed/PCP_Y2.csv", index_col=["MemberID"])

In [31]:
df_place_mode = pd.read_csv("./processed/PlaceSVCMode.csv", index_col=["MemberID"])

In [32]:
df_pcg_mode = pd.read_csv("./processed/PrimaryConditionGroupMode.csv", index_col=["MemberID"])

In [33]:
df_pg_mode = pd.read_csv("./processed/ProcedureGroup_Mode.csv", index_col=["MemberID"])
df_pg_y1 = pd.read_csv("./processed/ProcedureGroup_Y1.csv", index_col=["MemberID"])
df_pg_y2 = pd.read_csv("./processed/ProcedureGroup_Y2.csv", index_col=["MemberID"])

In [34]:
df_p_y1 = pd.read_csv("./processed/ProviderID_Y1.csv", index_col=["MemberID"])
df_p_y2 = pd.read_csv("./processed/ProviderID_Y2.csv", index_col=["MemberID"])

In [36]:
df_spe_y1 = pd.read_csv("./processed/Specialty_Y1.csv", index_col=["MemberID"])
df_spe_y2 = pd.read_csv("./processed/Specialty_Y2.csv", index_col=["MemberID"])
df_spe_mode = pd.read_csv("./processed/SpecialtyMode.csv", index_col=["MemberID"])

In [37]:
df_v_y1 = pd.read_csv("./processed/VendorID_Y1.csv", index_col=["MemberID"])
df_v_y2 = pd.read_csv("./processed/VendorID_Y2.csv", index_col=["MemberID"])

In [38]:
df_join = pd.concat([df_member, df_claim_count_y1, df_claim_count_y2, \
    df_char_y1, df_char_y2, df_drug_y1, df_drug_y2, df_dsfs_y1, df_dsfs_y2, \
        df_lab_y1, df_lab_y2, df_los_y1, df_los_y2, df_pay_max_y1, df_pay_max_y2, \
            df_pay_mean_y1, df_pay_mean_y2, df_pcp_y1, df_pcp_y2, df_place_mode, df_pcg_mode, \
                df_pg_mode, df_pg_y1, df_pg_y2, df_p_y1, df_p_y2, df_spe_y1, df_spe_y2, df_spe_mode, \
                    df_v_y1, df_v_y2], axis=1)

In [39]:
df_join

,AgeAtFirstClaim,Sex_M,Sex_NaN,ClaimCount_Y1,ClaimCount_Y2,CharlsonIndex_0,CharlsonIndex_1-2,CharlsonIndex_3-4,CharlsonIndex_5+,CharlsonIndex_0,...,ProcedureGroup_Mode,ProcedureGroup_Y1,ProcedureGroup_Y2,UniqueProviderID_Y1,UniqueProviderID_Y2,UniqueSpecialty_Y1,UniqueSpecialty_Y2,ModeSpeciaty,UniqueVendorID_Y1,UniqueVendorID_Y2
MemberID,,,,,,,,,,,,,,,,,,,,,
4,5.0,1,0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,EM,NaN,1.0,NaN,1.0,NaN,1.0,Pediatrics,NaN,1.0
210,35.0,0,1,8.0,6.0,8.0,0.0,0.0,0.0,6.0,...,MED,5.0,3.0,4.0,3.0,3.0,3.0,Internal,4.0,3.0
3197,5.0,0,0,5.0,5.0,5.0,0.0,0.0,0.0,5.0,...,EM,2.0,4.0,3.0,4.0,2.0,4.0,Pediatrics,3.0,3.0
3457,5.0,1,0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,EM,NaN,1.0,NaN,1.0,NaN,1.0,Pediatrics,NaN,1.0
3713,45.0,0,0,NaN,10.0,NaN,NaN,NaN,NaN,10.0,...,EM,NaN,3.0,NaN,5.0,NaN,3.0,General Practice,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996214,45.0,1,0,1.0,NaN,1.0,0.0,0.0,0.0,NaN,...,EM,1.0,NaN,1.0,NaN,1.0,NaN,General Practice,1.0,NaN
99997485,15.0,1,0,1.0,NaN,1.0,0.0,0.0,0.0,NaN,...,EM,1.0,NaN,1.0,NaN,1.0,NaN,General Practice,1.0,NaN
99997895,45.0,1,0,14.0,NaN,14.0,0.0,0.0,0.0,NaN,...,EM,4.0,NaN,5.0,NaN,4.0,NaN,Laboratory,4.0,NaN


In [40]:
len(set(df_join.index))

113000